In [4]:
import numpy as np
import pandas as pd

In [5]:
class FoundPrimers:
  def __init__(self, seq, maxTempDifference, minPrimer, maxPrimer):
    self.seq = seq 
    self.primerForward = ""
    self.primerReverse = ""
    self.primerReverseReversed = ""
    self.seqLen = len(seq)
    self.maxTempDifference = maxTempDifference
    self.minPrimer = minPrimer
    self.maxPrimer = maxPrimer
    self.matrix = np.array([])
    self.spaceForPrimers = min(self.seqLen-self.minPrimer, self.maxPrimer-self.minPrimer)
    self.min_i_forward = self.min_i_reverse = -1

    self.countMatrix()
    # self.printMatrix()
    # self.printBest()


  # 0 level - primer temp
  def l0_countTemp(self, seq):
    temp = (seq.count("G") + seq.count("C")) * 4 + (seq.count("T") + seq.count("A")) * 2
    return temp

  # 1 level - primers anti-Complementarity
  def l1_countComplementarity (self, seq1, seq2):
 
    if len(seq1) > len(seq2): 
      bigSeq = seq1
      smallSeq = seq2
    else:
      bigSeq = seq2
      smallSeq = seq1

    max = 0
    for type in ["normal", "reverse"]:
      if type == "reverse":
        smallSeq = smallSeq[::-1]
      for shift in range(len(smallSeq)):
        sum = 0
        for i in range(len(smallSeq)):
          if i+shift < len(bigSeq):
            #правда без награды за непрерывную комплиментарность(
            sum += self.complScore(smallSeq[i], bigSeq[i+shift])
      
        if sum > max: max = sum
    return max
        
        

  # 2 level - 2D structures
  def l2_countSelfComplementarity (self, seq):
    
    max = 0
    reverseSeq = seq[::-1]
    for shift in range(len(seq)):
      sum = 0
      for i in range(len(seq)):
        if i+shift < len(seq):
          #правда без награды за непрерывную комплиментарность(
          sum += self.complScore(seq[i], reverseSeq[i+shift])
    
      if sum > max: max = sum
    return max / 2


  # 3 level - G/C on 3' tale
  def l3_countGCtale(self, seq, type="forward"):
    char = seq[-1] if type == "forward" else seq[0]
    if (char in ["G", "C"]): return 0
    elif (char in ["A", "T"]): return 1
    else: return -1


  # 4 level - AT = GC
  def l4_ATequalsGC(self, seq):
    return abs(seq.count("G") + seq.count("C") - seq.count("A") - seq.count("T"))

  # 5 level - without GC regions
  def l5_countGCregions(self, seq):
    seq_1 = seq [:round(len(seq)/2)]
    seq_2 = seq [round(len(seq)/2):]
    return abs(seq_1.count("G") + seq_1.count("C") - seq_2.count("G") - seq_2.count("C"))
               

  # last level - sum of levels with priorities and minimisation
  def sumScore(self, tempDifference, complementarity, selfComplementarity, GCtale, ATequalsGC, GCregions):
    return tempDifference * 100 + complementarity * 10 +  selfComplementarity * 10 +  GCtale * 100 +  ATequalsGC * 10 +  GCregions * 10





  def complScore(self, a, b):
    if (a == "G" and b == "C") or (a == "C" and b == "G"): return 3
    elif (a == "A" and b == "T") or (a == "T" and b == "A"): return 2
    else: return 0

  def complement(self, seq):
    new_seq = ""
    for i in range(len(seq)):
      if seq[i] == "A":
        new_seq += "T"
      elif seq[i] == "T":
        new_seq += "A"
      elif seq[i] == "G":
        new_seq += "C"
      elif seq[i] == "C":
        new_seq += "G"
      else:
        print("error")
        return -1
    return new_seq

  def getSeq(self, i, type="forward"):
    if type == "forward":
      return self.seq[:self.minPrimer + i]
    elif type == "reverse":
      return self.seq[-(self.minPrimer + i):]
    else:
      print("error")
      return -1
  

  def countMatrix(self):
    if self.spaceForPrimers < 0:
      print("Too small sequence")
    else: 
      print("Ok")
      self.matrix = np.full((self.spaceForPrimers, self.spaceForPrimers, 7), 10000000)
      
      minScore = 10000000

      for i_forward in range(self.spaceForPrimers):
        seq_forward = self.getSeq(i_forward, "forward")

        for i_reverse in range(self.spaceForPrimers):
          seq_reverse = self.complement(self.getSeq(i_reverse, "reverse"))

          tempDifference = abs(self.l0_countTemp(seq_forward) - self.l0_countTemp(seq_reverse))
          self.matrix[i_forward, i_reverse, 0] = 0 if tempDifference <= self.maxTempDifference else tempDifference
          self.matrix[i_forward, i_reverse, 1] = self.l1_countComplementarity(seq_forward, seq_reverse)
          self.matrix[i_forward, i_reverse, 2] = max(self.l2_countSelfComplementarity(seq_forward), self.l2_countSelfComplementarity(seq_reverse))
          self.matrix[i_forward, i_reverse, 3] = self.l3_countGCtale(seq_forward, "forward") + self.l3_countGCtale(seq_reverse, "reverse")
          self.matrix[i_forward, i_reverse, 4] = max(self.l4_ATequalsGC(seq_forward), self.l4_ATequalsGC(seq_reverse))
          self.matrix[i_forward, i_reverse, 5] = max(self.l5_countGCregions(seq_forward), self.l5_countGCregions(seq_reverse))

          self.matrix[i_forward, i_reverse, 6] = self.sumScore(
              self.matrix[i_forward, i_reverse, 0],
              self.matrix[i_forward, i_reverse, 1],
              self.matrix[i_forward, i_reverse, 2],
              self.matrix[i_forward, i_reverse, 3],
              self.matrix[i_forward, i_reverse, 4],
              self.matrix[i_forward, i_reverse, 5],
          )
          
          if self.matrix[i_forward, i_reverse, 6] < minScore:
            self.min_i_forward = i_forward
            self.min_i_reverse = i_reverse
            minScore = self.matrix[i_forward, i_reverse, 6]

  
  def printMatrix(self): 
    print("\n0 level - primer temp")
    print(self.matrix[:,:,0])
    print("\n1 level - primers anti-Complementarity")
    print(self.matrix[:,:,1])
    print("\n2 level - 2D structures")
    print(self.matrix[:,:,2])
    print("\n3 level - G/C on 3' tale")
    print(self.matrix[:,:,3])
    print("\n4 level - AT = GC")
    print(self.matrix[:,:,4])
    print("\n5 level - without GC regions")
    print(self.matrix[:,:,5])
    print("\nlast level - sum of levels with priorities and minimisation")
    print(self.matrix[:,:,6])

  def printBest(self):
    result = self.matrix[self.min_i_forward, self.min_i_reverse, :]
    bestDict = {
        "sequence": self.seq,
        "sequenceCompl": self.complement(self.seq),
        "primerForward": self.getSeq(self.min_i_forward, "forward"),
        "primerReverce": self.complement(self.getSeq(self.min_i_reverse, "reverse")),
        "primerReverceReverced": self.complement(self.getSeq(self.min_i_reverse, "reverse"))[::-1],
        "tempForward": self.l0_countTemp(self.getSeq(self.min_i_forward, "forward")),
        "tempReverce": self.l0_countTemp(self.complement(self.getSeq(self.min_i_reverse, "reverse"))),
        "tempDifference": abs(self.l0_countTemp(self.getSeq(self.min_i_forward, "forward")) - self.l0_countTemp(self.complement(self.getSeq(self.min_i_reverse, "reverse")))),
        "complementarityScore": result[1],
        "selfComplementarityScore": result[2],
        "GCtaleScore": result[3],
        "ATequalsGC_Score": result[4],
        "GCregionsScore": result[5],
        "sumScore": result[6]
    }

    print("\n\nResults (less = better):")
    for i in bestDict:
      if i in ["sequence", "primerForward", "primerReverceReverced"]:
        print(i, ": 5\'", bestDict[i], "3\'")
      elif i in ["primerReverce", "sequenceCompl"]:
        print(i, ": 3\'", bestDict[i], "5\'")
      else:
        print(i, ": ", bestDict[i])

    return bestDict

In [6]:
seq = "GCTGATGCTTGATGATCGTCTCCATAGTCGTAGTAGCTGATGTCGTAGTAAAATGCTGATGCTGATAATCGTAGTA"
# seq = "dsadas"
maxTempDifference = 3
minPrimer = 15
maxPrimer = 25


primers = FoundPrimers(seq, maxTempDifference, minPrimer, maxPrimer)

Ok


In [7]:
primers.printMatrix()


0 level - primer temp
[[ 4  0  0  4  8 10 14 18 20 22]
 [ 6  0  0  0  6  8 12 16 18 20]
 [10  6  4  0  0  4  8 12 14 16]
 [14 10  8  6  0  0  4  8 10 12]
 [16 12 10  8  4  0  0  6  8 10]
 [20 16 14 12  8  6  0  0  4  6]
 [22 18 16 14 10  8  4  0  0  4]
 [26 22 20 18 14 12  8  4  0  0]
 [30 26 24 22 18 16 12  8  6  4]
 [32 28 26 24 20 18 14 10  8  6]]

1 level - primers anti-Complementarity
[[20 23 23 23 23 23 23 23 23 23]
 [20 23 23 23 23 23 23 23 23 23]
 [20 23 23 23 23 23 23 23 23 23]
 [20 23 20 23 23 23 23 23 23 23]
 [21 23 21 21 23 23 23 23 23 23]
 [21 23 21 21 21 23 23 23 23 23]
 [21 23 21 21 21 22 23 25 25 25]
 [21 23 21 21 21 22 25 25 25 25]
 [21 23 21 21 21 22 25 25 25 25]
 [21 23 21 21 21 22 25 25 25 25]]

2 level - 2D structures
[[ 7  7  9  9  9  9  9  9  9 11]
 [ 7  7  9  9  9  9  9  9  9 11]
 [ 9  9  9  9  9  9  9  9  9 11]
 [ 9  9  9  9  9  9  9  9  9 11]
 [ 9  9  9  9  9  9  9  9  9 11]
 [12 12 12 12 12 12 12 12 12 12]
 [12 12 12 12 12 12 12 12 12 12]
 [12 12 12 12 12 12

In [8]:
primers.printBest()



Results (less = better):
sequence : 5' GCTGATGCTTGATGATCGTCTCCATAGTCGTAGTAGCTGATGTCGTAGTAAAATGCTGATGCTGATAATCGTAGTA 3'
sequenceCompl : 3' CGACTACGAACTACTAGCAGAGGTATCAGCATCATCGACTACAGCATCATTTTACGACTACGACTATTAGCATCAT 5'
primerForward : 5' GCTGATGCTTGATGATCG 3'
primerReverce : 3' CTACGACTATTAGCATCAT 5'
primerReverceReverced : 5' TACTACGATTATCAGCATC 3'
tempForward :  54
tempReverce :  52
tempDifference :  2
complementarityScore :  23
selfComplementarityScore :  9
GCtaleScore :  0
ATequalsGC_Score :  5
GCregionsScore :  1
sumScore :  380


{'ATequalsGC_Score': 5,
 'GCregionsScore': 1,
 'GCtaleScore': 0,
 'complementarityScore': 23,
 'primerForward': 'GCTGATGCTTGATGATCG',
 'primerReverce': 'CTACGACTATTAGCATCAT',
 'primerReverceReverced': 'TACTACGATTATCAGCATC',
 'selfComplementarityScore': 9,
 'sequence': 'GCTGATGCTTGATGATCGTCTCCATAGTCGTAGTAGCTGATGTCGTAGTAAAATGCTGATGCTGATAATCGTAGTA',
 'sequenceCompl': 'CGACTACGAACTACTAGCAGAGGTATCAGCATCATCGACTACAGCATCATTTTACGACTACGACTATTAGCATCAT',
 'sumScore': 380,
 'tempDifference': 2,
 'tempForward': 54,
 'tempReverce': 52}